In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

TensorFlow version is  1.14.0-rc1


In [2]:
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip", 
                                   fname="cats_and_dogs_filtered.zip", extract=True)
base_dir, _ = os.path.splitext(zip_file)

68608000/68606236 [==============================] - 1s 0us/step


download filtered dataset for dogs and cats , and we will make directored for each subcategory in these

In [3]:
base_dir

'/root/.keras/datasets/cats_and_dogs_filtered'

In [4]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
print ('Total training cat images:', len(os.listdir(train_cats_dir)))

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
print ('Total training dog images:', len(os.listdir(train_dogs_dir)))

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print ('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

Total training cat images: 1000
Total training dog images: 1000
Total validation cat images: 500
Total validation dog images: 500


normalize the images , to be applicable with the model to be in standard format . to be (150,150)

In [0]:
image_size =150
patch_size = 30

traindatagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valedatedatagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [6]:
train_generator = traindatagen.flow_from_directory(train_dir ,target_size=(150,150),
                                                   batch_size=30 ,class_mode='binary')
valedate_generator = valedatedatagen.flow_from_directory(validation_dir ,target_size=(150,150),
                                                         batch_size=30 ,class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


we will not make any augmentaion steps here in this document , I want only to check the effect of the transfer learning in the dataset.

In [7]:
img_shape = (image_size , image_size , 3)
base_model = tf.keras.applications.MobileNetV2(input_shape =img_shape ,
                                               include_top =False ,weights ='imagenet')

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0618 08:37:32.995810 139926626260864 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


9412608/9406464 [==============================] - 1s 0us/step


In [8]:
base_model.trainable = False

base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

we will add the last two layers to classify the current job , because earlier layers concern only in the common shapes , edges , and things like that.

In [0]:
model = tf.keras.Sequential([base_model , keras.layers.GlobalAveragePooling2D(),
                           keras.layers.Dense(1 ,activation='sigmoid')])

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

W0618 08:37:47.176787 139926626260864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
len(model.trainable_variables)

2

In [13]:
epochs = 10
steps_per_epoch = train_generator.n
validation_steps = valedate_generator.n 

history = model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              validation_data=valedate_generator, 
                              validation_steps=validation_steps)

Epoch 1/10
2000/2000 [==============================] - 279s 139ms/step - loss: 0.2960 - acc: 0.8700 - val_loss: 0.2644 - val_acc: 0.9030
Epoch 2/10
2000/2000 [==============================] - 273s 136ms/step - loss: 0.1736 - acc: 0.9330 - val_loss: 0.2538 - val_acc: 0.9163
Epoch 3/10
2000/2000 [==============================] - 273s 137ms/step - loss: 0.1490 - acc: 0.9418 - val_loss: 0.2723 - val_acc: 0.9181
Epoch 4/10
2000/2000 [==============================] - 274s 137ms/step - loss: 0.1391 - acc: 0.9459 - val_loss: 0.3216 - val_acc: 0.9111
Epoch 5/10
2000/2000 [==============================] - 274s 137ms/step - loss: 0.1251 - acc: 0.9523 - val_loss: 0.3259 - val_acc: 0.9118
Epoch 6/10
2000/2000 [==============================] - 275s 138ms/step - loss: 0.1199 - acc: 0.9541 - val_loss: 0.3445 - val_acc: 0.9122
Epoch 7/10
2000/2000 [==============================] - 275s 137ms/step - loss: 0.1140 - acc: 0.9569 - val_loss: 0.3502 - val_acc: 0.9131
Epoch 8/10
2000/2000 [============

we will notice that the accuracy has moved up to be 96% percent and the validation accurace moved to be 91 % and this is high degree .